In [1]:
import graphlab as gl

In [46]:
sf = gl.SFrame({'user_id': ["Marvin", "Marvin", "Brian", "Brian", "Jon", "Jon", "Eugene", "Eugene"],
                'item_id': ["french", "thai", "italian", "thai", "thai", "french", "french", "italian"],
                'rating': [1., 3., 2., 5., 4., 1., 4., 3.]})

In [47]:
sf

item_id,rating,user_id
french,1.0,Marvin
thai,3.0,Marvin
italian,2.0,Brian
thai,5.0,Brian
thai,4.0,Jon
french,1.0,Jon
french,4.0,Eugene
italian,3.0,Eugene


In [48]:
group_list = ["Marvin", "Brian"]
group_name = "_".join(group_list)

In [49]:
sf_group = sf.filter_by(group_list, "user_id") \
             .groupby(key_columns='item_id',
                      operations={'rating': gl.aggregate.MEAN('rating')})  

In [50]:
sf_group.add_column(gl.SArray([group_name] * len(sf_group)), "user_id")

item_id,rating,user_id
french,1.0,Marvin_Brian
italian,2.0,Marvin_Brian
thai,4.0,Marvin_Brian


In [51]:
sf_new = sf.append(sf_group)

In [52]:
sf_new

item_id,rating,user_id
french,1.0,Marvin
thai,3.0,Marvin
italian,2.0,Brian
thai,5.0,Brian
thai,4.0,Jon
french,1.0,Jon
french,4.0,Eugene
italian,3.0,Eugene
french,1.0,Marvin_Brian
italian,2.0,Marvin_Brian


In [53]:
sf_new["user_id"].unique()

dtype: str
Rows: 5
['Jon', 'Marvin', 'Brian', 'Marvin_Brian', 'Eugene']

In [54]:
model = gl.recommender.create(sf_new, target='rating')

Recsys training: model = ranking_factorization_recommender

Preparing data set.

Data has 11 observations with 5 users and 3 items.

Data prepared in: 0.005121s

Training ranking_factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 1.75244

Final training RMSE: 0.530101

In [60]:
results = model.recommend(["Marvin_Brian"], exclude_known=False)

In [66]:
result_cuisine = results["item_id"][0]

In [67]:
data = {"thai": sf_thai}

In [68]:
data[result_cuisine]

item_id,rating,user_id
pad_thai,1.0,Marvin
kow_soi,3.0,Marvin
spring_rolls,2.0,Brian
kow_soi,5.0,Brian
pad_thai,4.0,Jon
kow_soi,1.0,Jon
spring_rolls,4.0,Eugene
pad_thai,3.0,Eugene


In [56]:
sf_thai = gl.SFrame({'user_id': ["Marvin", "Marvin", "Brian", "Brian", "Jon", "Jon", "Eugene", "Eugene"],
                'item_id': ["pad_thai", "kow_soi", "spring_rolls", "kow_soi", "pad_thai", "kow_soi", "spring_rolls", "pad_thai"],
                'rating': [1., 3., 2., 5., 4., 1., 4., 3.]})

In [57]:
sf_thai

item_id,rating,user_id
pad_thai,1.0,Marvin
kow_soi,3.0,Marvin
spring_rolls,2.0,Brian
kow_soi,5.0,Brian
pad_thai,4.0,Jon
kow_soi,1.0,Jon
spring_rolls,4.0,Eugene
pad_thai,3.0,Eugene


In [58]:
model_thai = gl.recommender.create(sf_thai, target='rating')

Recsys training: model = ranking_factorization_recommender

Preparing data set.

Data has 8 observations with 4 users and 3 items.

Data prepared in: 0.003414s

Training ranking_factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 3.10345

Final training RMSE: 1.18332

In [69]:
results_items = model_thai.recommend(group_list)

In [91]:
results_items[["user_id", "item_id"]].to_numpy().tolist()

[['Marvin', 'spring_rolls'], ['Brian', 'pad_thai']]